### Preparación de datos

In [1]:
from pathlib import Path
import pandas as pd

# === RUTAS ===

ruta_acta =  Path(r"20251003/20251003/ADP_DTM_FACT.Acta.csv")
ruta_insumos =  Path(r"20251003/20251003/ADP_DTM_DIM.Insumo.csv")
ruta_proyectos =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")
ruta_capitulos = Path(r"20251003/20251003/ADP_DTM_DIM.CapituloPresupuesto.csv")
ruta_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")




In [2]:
# === CARGA ===
tabla_proyeccion = pd.read_csv(ruta_proyeccion)
tabla_items = pd.read_csv(ruta_items)
tabla_proyectos = pd.read_csv(ruta_proyectos)
tabla_capitulos = pd.read_csv(ruta_capitulos)
tabla_insumos = pd.read_csv(ruta_insumos)

C:\Users\aleja\AppData\Local\Temp\ipykernel_6912\2920138097.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  tabla_items = pd.read_csv(ruta_items)


### Un join

In [7]:
tabla_base = tabla_proyeccion.copy()

# Merges previos
tabla_1 = pd.merge(tabla_base, tabla_proyectos, on="SkIdProyecto", how="left")
tabla_2 = pd.merge(tabla_1, tabla_capitulos, on="SkIdCapitulo", how="left")

# Asegura unicidad en Items
tabla_items_unica = tabla_items.drop_duplicates(subset=["SkIdItems"], keep="first")
tabla_3 = pd.merge(
    tabla_2, tabla_items_unica,
    on="SkIdItems", how="left",
    suffixes=("", "_item")   # control de sufijos para items
)

# 🔧 FIX insumos: prefijar y garantizar clave única
tabla_insumos_unica = tabla_insumos.drop_duplicates(subset=["SkIdInsumo"], keep="first").copy()
cols_no_clave_insumo = [c for c in tabla_insumos_unica.columns if c != "SkIdInsumo"]
tabla_insumos_pref = tabla_insumos_unica.rename(
    columns={c: f"Insumo_{c}" for c in cols_no_clave_insumo}
)

# (Opcional) Debug rápido de posibles overlaps antes del merge final
overlap_prev = set(tabla_3.columns).intersection(set(tabla_insumos_pref.columns))
print(f"[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {overlap_prev}")

# Merge final (sin sufijos porque ya prefijamos)
tabla_4 = pd.merge(
    tabla_3, tabla_insumos_pref,
    on="SkIdInsumo", how="left",
    validate="many_to_one"
)

print(f"[DEBUG] OK. Forma final: {tabla_4.shape}")

tabla_4.to_csv(
    "tabla_looker.csv",
    index=False,
    encoding="utf-8",
    sep=","                
)

[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {'SkIdInsumo'}
[DEBUG] OK. Forma final: (273450, 107)


KeyboardInterrupt: 

In [8]:
# === SELECCIÓN DE COLUMNAS ===
columnas_finales = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "Nombre Proyecto", "Capitulo Descripcion", "Item Descripcion", "Insumo_Insumo Descripcion", "Insumo_Agrupacion Descripcion"
    "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total", "Insumo_Valor Unitario", "Insumo_Valor Neto", "Insumo_Fecha Creacion",
    "Cantidad Item", "Macroproyecto Descripcion", "Insumo_Fecha Modificacion",
    "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalización", "SkIdFecha", "Capitulo Numero", "Cantidad_Item"
]

# Filtrar solo las columnas que existan realmente (por seguridad)
columnas_existentes = [col for col in columnas_finales if col in tabla_4.columns]
tabla_looker = tabla_4[columnas_existentes].copy()

#### Debug

# Parte dos tabla final 

In [ ]:
import pandas as pd
from datetime import datetime

# === RUTA DE SALIDA ===
ruta_salida = r"Tabla_Final_ARPRO_Completa.csv"

# === 1) PROYECCION ===
proyeccion = pd.read_csv(ruta_proyeccion, low_memory=False)
columnas_proyeccion = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "SkIdFecha", "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total"
]
missing = [c for c in columnas_proyeccion if c not in proyeccion.columns]
if missing:
    raise KeyError(f"Faltan columnas en Proyeccion: {missing}")

proyeccion_filtrada = proyeccion[columnas_proyeccion].rename(columns={
    "Cantidad": "Cantidad_Insumo",
    "Valor Unitario": "Valor_Unitario_Insumo",
    "Valor Total": "Valor_Total_Item"
})

# === 2) PROYECTO ===
proyecto = pd.read_csv(ruta_proyectos, low_memory=False)
columnas_proyecto = [
    "SkIdProyecto", "Codigo Proyecto", "Nombre Proyecto", "Clase Proyecto", "Estado",
    "MacroProyecto Descripcion", "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalizacion"
]
missing = [c for c in columnas_proyecto if c not in proyecto.columns]
if missing:
    raise KeyError(f"Faltan columnas en Proyecto: {missing}")

proyecto_filtrado = proyecto[columnas_proyecto].rename(columns={
    "Codigo Proyecto": "Codigo_Proyecto",
    "Nombre Proyecto": "Nombre_Proyecto",
    "Clase Proyecto": "Clase_Proyecto",
    "Estado": "Estado_Proyecto",
    "MacroProyecto Descripcion": "MacroProyecto_Descripcion_Proyecto",
    "Fecha De Elaboracion": "Fecha_De_Elaboracion_Proyecto",
    "Fecha De Inicio": "Fecha_De_Inicio_Proyecto",
    "Fecha De Finalizacion": "Fecha_De_Finalizacion_Proyecto"
})

merge_proyeccion_proyecto = pd.merge(
    proyeccion_filtrada, proyecto_filtrado,
    on="SkIdProyecto", how="left", validate="m:1"
)
print(f"DimP: {proyeccion.shape}\n Dim")
merge_proyeccion_proyecto

In [10]:
import pandas as pd
from datetime import datetime

# === RUTA DE SALIDA ===
ruta_salida = r"Tabla_Final_ARPRO_Completa.csv"

# === 1) PROYECCION ===
proyeccion = pd.read_csv(ruta_proyeccion, low_memory=False)
columnas_proyeccion = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "SkIdFecha", "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total"
]
missing = [c for c in columnas_proyeccion if c not in proyeccion.columns]
if missing:
    raise KeyError(f"Faltan columnas en Proyeccion: {missing}")

proyeccion_filtrada = proyeccion[columnas_proyeccion].rename(columns={
    "Cantidad": "Cantidad_Insumo",
    "Valor Unitario": "Valor_Unitario_Insumo",
    "Valor Total": "Valor_Total_Item"
})

# === 2) PROYECTO ===
proyecto = pd.read_csv(ruta_proyectos, low_memory=False)
columnas_proyecto = [
    "SkIdProyecto", "Codigo Proyecto", "Nombre Proyecto", "Clase Proyecto", "Estado",
    "MacroProyecto Descripcion", "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalizacion"
]
missing = [c for c in columnas_proyecto if c not in proyecto.columns]
if missing:
    raise KeyError(f"Faltan columnas en Proyecto: {missing}")

proyecto_filtrado = proyecto[columnas_proyecto].rename(columns={
    "Codigo Proyecto": "Codigo_Proyecto",
    "Nombre Proyecto": "Nombre_Proyecto",
    "Clase Proyecto": "Clase_Proyecto",
    "Estado": "Estado_Proyecto",
    "MacroProyecto Descripcion": "MacroProyecto_Descripcion_Proyecto",
    "Fecha De Elaboracion": "Fecha_De_Elaboracion_Proyecto",
    "Fecha De Inicio": "Fecha_De_Inicio_Proyecto",
    "Fecha De Finalizacion": "Fecha_De_Finalizacion_Proyecto"
})

merge_proyeccion_proyecto = pd.merge(
    proyeccion_filtrada, proyecto_filtrado,
    on="SkIdProyecto", how="left", validate="m:1"
)

# === 3) ITEMS ===
items = pd.read_csv(ruta_items, low_memory=False)
columnas_items = ["SkIdItems", "Item Descripcion"]
missing = [c for c in columnas_items if c not in items.columns]
if missing:
    raise KeyError(f"Faltan columnas en Items: {missing}")

items_filtrado = items[columnas_items].rename(columns={"Item Descripcion": "Item_Descripcion"})

merge_items = pd.merge(
    merge_proyeccion_proyecto, items_filtrado,
    on="SkIdItems", how="left", validate="m:1"
)

# === 4) INSUMO ===
insumo = pd.read_csv(ruta_insumos, low_memory=False)
columnas_insumo = ["SkIdInsumo", "Insumo Descripcion", "Fecha Creacion", "Fecha Modificacion"]
missing = [c for c in columnas_insumo if c not in insumo.columns]
if missing:
    raise KeyError(f"Faltan columnas en Insumo: {missing}")

insumo_filtrado = insumo[columnas_insumo].rename(columns={
    "Insumo Descripcion": "Insumo_Descripcion",
    "Fecha Creacion": "Fecha_Creacion_Insumo",
    "Fecha Modificacion": "Fecha_Modificacion_Insumo"
})

tabla_intermedia = pd.merge(
    merge_items, insumo_filtrado,
    on="SkIdInsumo", how="left", validate="m:1"
)

# === 5) CAPITULO PRESUPUESTO ===
capitulo_presupuesto = pd.read_csv(ruta_capitulos, low_memory=False)
columnas_capitulo = ["Codigo Proyecto", "Capitulo Descripcion"]
missing = [c for c in columnas_capitulo if c not in capitulo_presupuesto.columns]
if missing:
    raise KeyError(f"Faltan columnas en CapituloPresupuesto: {missing}")

capitulo_filtrado = capitulo_presupuesto[columnas_capitulo].rename(columns={
    "Codigo Proyecto": "Codigo_Proyecto",
    "Capitulo Descripcion": "Capitulo_Descripcion"
})

if "Codigo_Proyecto" not in tabla_intermedia.columns:
    proyecto_cod = proyecto_filtrado[["SkIdProyecto", "Codigo_Proyecto"]].drop_duplicates()
    tabla_intermedia = pd.merge(
        tabla_intermedia, proyecto_cod,
        on="SkIdProyecto", how="left", validate="m:1"
    )

tabla_final = pd.merge(
    tabla_intermedia, capitulo_filtrado,
    on="Codigo_Proyecto", how="left", validate="m:m"
)

# === 6) RELLENAR VALORES FALTANTES ===
tabla_final["MacroProyecto_Descripcion_Proyecto"] = tabla_final["MacroProyecto_Descripcion_Proyecto"].fillna("No pertenece a un Macro Proyecto")
tabla_final["Insumo_Descripcion"] = tabla_final["Insumo_Descripcion"].fillna("Descripcion No Disponible")
tabla_final["Fecha_Modificacion_Insumo"] = tabla_final["Fecha_Modificacion_Insumo"].fillna("2009-12-31")
tabla_final["Fecha_Creacion_Insumo"] = tabla_final["Fecha_Creacion_Insumo"].fillna("2009-12-31")

# === 7) CONVERTIR FECHAS A FORMATO ISO ===
def convertir_fecha(columna):
    """
    Convierte diferentes formatos de fecha al estándar ISO (YYYY-MM-DD)
    """
    fechas = pd.to_datetime(columna, errors="coerce", dayfirst=True)
    return fechas.dt.strftime("%Y-%m-%d")

tabla_final["Fecha_Creacion_Insumo"] = convertir_fecha(tabla_final["Fecha_Creacion_Insumo"])
tabla_final["Fecha_Modificacion_Insumo"] = convertir_fecha(tabla_final["Fecha_Modificacion_Insumo"])

# Reportar errores
for col in ["Fecha_Creacion_Insumo", "Fecha_Modificacion_Insumo"]:
    errores = tabla_final[col].isna().sum()
    print(f"🧮 {col}: {errores} valores no convertidos (NaT)")

print("\n✅ Fechas unificadas en formato YYYY-MM-DD.")


🧮 Fecha_Creacion_Insumo: 93 valores no convertidos (NaT)
🧮 Fecha_Modificacion_Insumo: 1401963 valores no convertidos (NaT)

✅ Fechas unificadas en formato YYYY-MM-DD.


In [ ]:

# === 8) GUARDAR RESULTADO FINAL ===
tabla_final.to_csv(ruta_salida, index=False)
print("\n✅ Proceso completado. Archivo guardado en:", ruta_salida)